In [1]:
import os
import pandas as pd
import networkx as nx
import numpy as np
from networkx.utils import reverse_cuthill_mckee_ordering

In [2]:
NUMBER_NODES = 9

def load_data():
    test_df = pd.read_csv(os.path.join('datasets', f'dataset_{NUMBER_NODES}_test.csv'))

    featuresNumber = (NUMBER_NODES * NUMBER_NODES - NUMBER_NODES) // 2 
    def get_tuple_tensor_dataset(row):
        X = row[0 : featuresNumber].astype('int32')
        Y = row[featuresNumber + 1: ].astype('int32') # Pula a banda otima na posicao 0
        return X, Y

    test_dataset = list(map(get_tuple_tensor_dataset, test_df.to_numpy()))

    X = []
    Y = []
    for x, y in test_dataset:
        X.append(x)
        Y.append(y)
    x_test = np.array(X)
    y_test = np.array(Y)

    return x_test, y_test

In [3]:
x_test, y_test = load_data()

In [4]:
def get_optimal_bandwidth_nodelist_adjacency_rcm(Graph):
    rcm = list(reverse_cuthill_mckee_ordering(Graph))
    A = nx.adjacency_matrix(Graph, nodelist=rcm)
    L = nx.laplacian_matrix(nx.Graph(A))
    x, y = np.nonzero(L)
    return (x-y).max()

In [5]:
def count_repeats(output):
    counts = np.unique(np.round(output))
    repeated = NUMBER_NODES - counts.shape[0]
    return repeated

def get_valid_pred(pred):
    valid = np.ones(NUMBER_NODES)
    labels = np.arange(0, NUMBER_NODES)
    for i in labels:
        min_value = np.amin(pred)
        min_idx, = np.where(pred == min_value)
        min_idx = min_idx[0]
        pred[min_idx] = 100
        valid[min_idx] = i
    return valid
    
def getGraph(upperTriangleAdjMatrix):
    dense_adj = np.zeros((NUMBER_NODES, NUMBER_NODES))
    dense_adj = np.zeros((NUMBER_NODES, NUMBER_NODES))
    k = 0
    for i in range(NUMBER_NODES):
        for j in range(NUMBER_NODES):
            if i == j:
                continue
            elif i < j:
                dense_adj[i][j] = upperTriangleAdjMatrix[k]
                k += 1
            else:
                dense_adj[i][j] = dense_adj[j][i]
    return dense_adj

def get_bandwidth(Graph, nodelist=None):
    if not Graph.edges:
        return 0
    if nodelist.all() != None:
        L = nx.laplacian_matrix(Graph, nodelist=nodelist)
    else:
        L = nx.laplacian_matrix(Graph)
    x, y = np.nonzero(L)
    return (x-y).max()

In [6]:
import time

sumTest_original = []
sumTest_pred = []
sumTest_true = []

count = 0
cases_with_repetition = 0

start = time.time()
for i in range(len(x_test)):
    graph = getGraph(x_test[i])
    Graph = nx.Graph(graph)
    output = np.array(list(reverse_cuthill_mckee_ordering(Graph)))

    quantity_repeated = count_repeats(np.round(output))

    # print('Pred: ', output)
    # print('True: ', y_test[i])

    if quantity_repeated != 0:
        cases_with_repetition += 1

    output = get_valid_pred(output)
    # print('Pred valid: ', output)
    count += quantity_repeated

    original_band = get_bandwidth(Graph, np.array(None))
    sumTest_original.append(original_band)

    pred_band = get_bandwidth(Graph, output)
    sumTest_pred.append(pred_band)

    true_band = get_bandwidth(Graph, y_test[i])
    sumTest_true.append(true_band)
    # print("Bandwidth")
    # print(original_band)
    # print(pred_band)
    # print(true_band)
end = time.time()

In [7]:
print('Quantidade de rótulos repetidos, exemplo [1, 1, 1, 1, 1, 1, 1] conta como 6 - ', count)
print('Quantidade de saídas com repetição, exemplo [1, 1, 1, 1, 1, 1, 1] conta como 1 - ', cases_with_repetition)
test_length = x_test.shape[0]
print('Test length - ', test_length)
print('Time- ', (end - start) / test_length)

print("Bandwidth mean±std")
print(f'{np.mean(sumTest_original)}±{np.std(sumTest_original)}')
print("Pred bandwidth mean±std")
print(f'{np.mean(sumTest_pred)}±{np.std(sumTest_pred)}')
print("True bandwidth mean±std")
print(f'{np.mean(sumTest_true)}±{np.std(sumTest_true)}')

Quantidade de rótulos repetidos, exemplo [1, 1, 1, 1, 1, 1, 1] conta como 6 -  0
Quantidade de saídas com repetição, exemplo [1, 1, 1, 1, 1, 1, 1] conta como 1 -  0
Test length -  16480
Time-  0.004805473390134793
Bandwidth mean±std
7.7975728155339805±0.4248574084483311
Pred bandwidth mean±std
4.775±0.8954340760022591
True bandwidth mean±std
4.092839805825243±0.6732562471584104
